In [1]:
import requests
import pandas as pd
import time
import os
from config import token

Define API Function to fetch data and save as csv.

In [10]:
def fetch_save_data(url, headers, params, csv_filename):
    results = []
    
    while True:
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['results'])
            results.append(df)
            
            if len(df) < params['limit']:
                break
            
            params['offset'] += params['limit']
        else:
            print("Error:", response.status_code)
            break
    
    df_name = pd.concat(results, ignore_index=True)
    response_count = df_name.shape[0]
    print(f"Response Count: {response_count}")
    
    df_name.to_csv(csv_filename, index=False)
    return df_name


Pull Datatypes
*note: I've been getting a 503 (server busy or unavailable) error at between 60k-120k rows. I've had sucess trying again later or split the data into smaller sections and combine

In [11]:
# Define the API endpoint URL, headers, pagination parameters, and CSV filename
api_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes"
api_headers = {"token": token}
api_params = {'offset': 0, 
              'limit': 1000
             }
output_csv_filename = 'Outputs/datatypes.csv'

# Call the function with the defined values
datatypes_df = fetch_save_data(api_url, api_headers, api_params, output_csv_filename)

# Display the first few rows of the DataFrame
datatypes_df.head()


Response Count: 1567


,mindate,maxdate,name,datacoverage,id
0,1994-03-19,1996-05-28,Average cloudiness midnight to midnight from 3...,1.00,ACMC
1,1965-01-01,2005-12-31,Average cloudiness midnight to midnight from m...,1.00,ACMH
2,1994-02-01,1996-05-28,Average cloudiness sunrise to sunset from 30-s...,1.00,ACSC
3,1965-01-01,2005-12-31,Average cloudiness sunrise to sunset from manu...,1.00,ACSH
4,1991-06-05,2023-08-03,Base Data,0.95,ALL


Pull Station List

In [31]:
#Sometimes you have to try and run this several times to get all data...not complete if showing 503 error

# Define the API endpoint URL, headers, pagination parameters, and CSV filename
api_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/stations"
api_headers = {"token": token}
api_params = {'offset': 0, 
              'limit': 1000
             }
output_csv_filename = 'Outputs/full_station_list.csv'

# Call the function with the defined values
stations_df = fetch_save_data(api_url, api_headers, api_params, output_csv_filename)

# Display the first few rows of the DataFrame
stations_df.head()


Response Count: 149412


,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
0,139.0,1948-01-01,2014-01-01,31.57020,"ABBEVILLE, AL US",0.8813,COOP:010008,METERS,-85.24820
1,239.6,1938-01-01,2015-11-01,34.21096,"ADDISON, AL US",0.5059,COOP:010063,METERS,-87.17838
2,302.1,1940-05-01,1962-03-01,34.41667,"ADDISON CENTRAL TOWER, AL US",0.9658,COOP:010071,METERS,-87.31667
3,172.5,1995-04-01,2015-11-01,33.17835,"ALABASTER SHELBY CO AIRPORT ASOS, AL US",0.8064,COOP:010116,METERS,-86.78178
4,183.8,1949-01-01,1949-12-01,34.68910,"BELLE MINA 2 N, AL US",1.0000,COOP:010117,METERS,-86.88190


Pull Data Categories

In [14]:
# Define the API endpoint URL, headers, pagination parameters, and CSV filename
api_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/datacategories"
api_headers = {"token": token}
api_params = {'offset': 0, 
              'limit': 1000
             }
output_csv_filename = 'Outputs/data_categories.csv'

# Call the function with the defined values
data_categories_df = fetch_save_data(api_url, api_headers, api_params, output_csv_filename)

# Display the first few rows of the DataFrame
data_categories_df.head()

Response Count: 42


,name,id
0,Annual Agricultural,ANNAGR
1,Annual Degree Days,ANNDD
2,Annual Precipitation,ANNPRCP
3,Annual Temperature,ANNTEMP
4,Autumn Agricultural,AUAGR


Pull Available Datasets

In [15]:
# Define the API endpoint URL, headers, pagination parameters, and CSV filename
api_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/datasets"
api_headers = {"token": token}
api_params = {'offset': 0, 
              'limit': 1000
             }
output_csv_filename = 'Outputs/datasets.csv'

# Call the function with the defined values
datasets_df = fetch_save_data(api_url, api_headers, api_params, output_csv_filename)

# Display the first few rows of the DataFrame
datasets_df.head(11)


Response Count: 11


,uid,mindate,maxdate,name,datacoverage,id
0,gov.noaa.ncdc:C00861,1763-01-01,2023-08-01,Daily Summaries,1.00,GHCND
1,gov.noaa.ncdc:C00946,1763-01-01,2023-07-01,Global Summary of the Month,1.00,GSOM
2,gov.noaa.ncdc:C00947,1763-01-01,2023-01-01,Global Summary of the Year,1.00,GSOY
3,gov.noaa.ncdc:C00345,1991-06-05,2023-08-03,Weather Radar (Level II),0.95,NEXRAD2
4,gov.noaa.ncdc:C00708,1994-05-20,2023-08-01,Weather Radar (Level III),0.95,NEXRAD3
5,gov.noaa.ncdc:C00821,2010-01-01,2010-01-01,Normals Annual/Seasonal,1.00,NORMAL_ANN
6,gov.noaa.ncdc:C00823,2010-01-01,2010-12-31,Normals Daily,1.00,NORMAL_DLY
7,gov.noaa.ncdc:C00824,2010-01-01,2010-12-31,Normals Hourly,1.00,NORMAL_HLY
8,gov.noaa.ncdc:C00822,2010-01-01,2010-12-01,Normals Monthly,1.00,NORMAL_MLY
9,gov.noaa.ncdc:C00505,1970-05-12,2014-01-01,Precipitation 15 Minute,0.25,PRECIP_15


Pull Texas Temperature Data from 1923 to 2023 (in 5 year increments)

In [22]:
# Define the API endpoint URL, headers, pagination parameters, and CSV filename
api_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
api_headers = {"token": token}

# Set up pagination parameters
offset = 0
limit = 1000
datasetid = 'GSOM'
datatypeids = ['TAVG', 'TMAX', 'TMIN']
startdate = '1923-08-01'
enddate = '1933-07-31'
locationid = 'FIPS:48'  # Texas FIPS code

api_params = {
    'datasetid': datasetid,
    'startdate': startdate,
    'locationid': locationid,
    'enddate': enddate,
    'offset': offset,
    'limit': limit
}

# Combine datatypeids into a comma-separated string
api_params['datatypeid'] = ','.join(datatypeids)

output_csv_filename = 'Outputs/tx_station_temps.csv'

# Call the function with the defined values
tx_station_temps_df = fetch_save_data(api_url, api_headers, api_params, output_csv_filename)

# Display the first few rows of the DataFrame
tx_station_temps_df.tail(10)

Response Count: 44207


,date,datatype,station,attributes,value
44197,1933-07-01T00:00:00,TMIN,GHCND:USW00023080,",,,X",22.45
44198,1933-07-01T00:00:00,TAVG,GHCND:USW00093914,",0",28.12
44199,1933-07-01T00:00:00,TMAX,GHCND:USW00093914,",,,0",33.15
44200,1933-07-01T00:00:00,TMIN,GHCND:USW00093914,",,,0",23.09
44201,1933-07-01T00:00:00,TAVG,GHCND:USW00093928,",X",28.71
44202,1933-07-01T00:00:00,TMAX,GHCND:USW00093928,",,,X",33.46
44203,1933-07-01T00:00:00,TMIN,GHCND:USW00093928,",,,X",23.96
44204,1933-07-01T00:00:00,TAVG,GHCND:USW00093987,",0",28.80
44205,1933-07-01T00:00:00,TMAX,GHCND:USW00093987,",,,0",34.85
44206,1933-07-01T00:00:00,TMIN,GHCND:USW00093987,",,,0",22.76


Combine Texas Temperature Data and Remove Duplicates

In [24]:
# Folder containing the CSV files
folder = 'Outputs/texas_temp_data'

# List of CSV filenames
filenames = ['1923-1933_tx_station_temps.csv','1933-1938_tx_station_temps.csv', '1938-1943_tx_station_temps.csv', '1943-1948_tx_station_temps.csv', 
             '1948-1953_tx_station_temps.csv', '1953-1958_tx_station_temps.csv', '1958-1963_tx_station_temps.csv',
             '1963-1968_tx_station_temps.csv', '1968-1973_tx_station_temps.csv', '1973-1978_tx_station_temps.csv',
             '1978-1983_tx_station_temps.csv', '1983-1988_tx_station_temps.csv', '1988-1993_tx_station_temps.csv', 
             '1993-1998_tx_station_temps.csv', '1998-2003_tx_station_temps.csv', '2003-2008_tx_station_temps.csv',
             '2008-2013_tx_station_temps.csv','2013-ext_tx_station_temps.csv','2013-2018_tx_station_temps.csv',
             '2018-2023_tx_station_temps.csv']

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Loop through the filenames and read each CSV into a DataFrame
for filename in filenames:
    file_path = os.path.join(folder, filename)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    else:
        print(f"File not found: {file_path}")

# Remove duplicates from the combined DataFrame
combined_df = combined_df.drop_duplicates()

# Save the combined and deduplicated DataFrame to a new CSV file
output_filename = os.path.join('Outputs/combined_temps.csv')
combined_df.to_csv(output_filename, index=False)

print(f"Combined CSVs and removed duplicates. Saved as '{output_filename}'.")


Combined CSVs and removed duplicates. Saved as 'Outputs/combined_temps.csv'.


Consolidate Min/Max/Avg Temp Data on One Row

In [26]:
# Read the data from your CSV or Excel file
file_path = 'Outputs/combined_temps.csv'
data_df = pd.read_csv(file_path)

# Initialize empty lists to store the data
dates = []
stations = []
tmax_values = []
tmin_values = []
tavg_values = []

# Iterate through the rows of the original DataFrame
for index, row in data_df.iterrows():
    date = row['date']
    station = row['station']
    datatype = row['datatype']
    value = row['value']
    
    if datatype == 'TMAX':
        tmax_values.append(value)
        tmin_values.append(None)
        tavg_values.append(None)
    elif datatype == 'TMIN':
        tmax_values.append(None)
        tmin_values.append(value)
        tavg_values.append(None)
    elif datatype == 'TAVG':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(value)
    
    dates.append(date)
    stations.append(station)

# Create a new DataFrame
new_data = {
    'date': dates,
    'station': stations,
    'TMAX': tmax_values,
    'TMIN': tmin_values,
    'TAVG': tavg_values
}

new_df = pd.DataFrame(new_data)

# Group by date and station and keep non-null values
grouped_df = new_df.groupby(['date', 'station']).first().reset_index()

# Write the grouped DataFrame to a CSV file
csv_filename = 'Outputs/grouped_df.csv'
grouped_df.to_csv(csv_filename, index=False)

# Print the first few rows of the grouped DataFrame
grouped_df.head()


,date,station,TMAX,TMIN,TAVG
0,1923-08-01T00:00:00,GHCND:USC00296119,33.87,21.66,27.76
1,1923-08-01T00:00:00,GHCND:USC00410120,37.83,21.94,29.89
2,1923-08-01T00:00:00,GHCND:USC00410144,36.78,23.47,30.13
3,1923-08-01T00:00:00,GHCND:USC00410257,33.51,22.15,27.83
4,1923-08-01T00:00:00,GHCND:USC00410420,39.47,21.71,30.59


Add Station Details to grouped_df

In [33]:
# Load the stations.csv file into stations_df
stations_df = pd.read_csv('Outputs/full_station_list.csv')
grouped_df= pd.read_csv('Outputs/grouped_df.csv')

# Merge the two DataFrames based on 'station' using a left join
grouped_df_detailed = pd.merge(grouped_df, stations_df, 
                               left_on='station', right_on='id', how='left')

# Drop the redundant columns (from stations_df)
grouped_df_detailed.drop(columns=['id'], inplace=True)

# Write the grouped DataFrame to a CSV file
csv_filename = 'Outputs/grouped_df_detailed.csv'
grouped_df_detailed.to_csv(csv_filename, index=False)

grouped_df_detailed.head()

,date,station,TMAX,TMIN,TAVG,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude
0,1923-08-01T00:00:00,GHCND:USC00296119,33.87,21.66,27.76,1215.8,1909-07-01,1933-03-31,32.00000,"NEWMAN, TX US",1.0000,METERS,-106.31667
1,1923-08-01T00:00:00,GHCND:USC00410120,37.83,21.94,29.89,441.4,1893-01-01,2023-07-31,32.70470,"ALBANY, TX US",1.0000,METERS,-99.30120
2,1923-08-01T00:00:00,GHCND:USC00410144,36.78,23.47,30.13,61.3,1893-01-01,2010-12-31,27.72833,"ALICE, TX US",1.0000,METERS,-98.06778
3,1923-08-01T00:00:00,GHCND:USC00410257,33.51,22.15,27.83,8.2,1895-06-01,2023-03-31,29.15730,"ANGLETON 2 W, TX US",1.0000,METERS,-95.45930
4,1923-08-01T00:00:00,GHCND:USC00410420,39.47,21.71,30.59,180.1,1893-02-01,1942-07-31,30.28333,"AUSTIN, TX US",0.9614,METERS,-97.73333


In [34]:
grouped_df_detailed.shape

(369788, 13)

Get Count of Data Points per Station and Texas Station Summary

In [4]:
grouped_df_detailed = pd.read_csv('Outputs/grouped_df_detailed.csv')

station_counts=grouped_df_detailed['station'].value_counts().reset_index()

# Rename the columns in the station_counts DataFrame
station_counts.columns = ['station', 'count']
station_counts=station_counts.sort_values('count',ascending=False)

# Add 'station' and 'name' columns from grouped_df_detailed
  
station_counts = pd.merge(station_counts, grouped_df_detailed[['station','name','maxdate','mindate','longitude','latitude','elevation']], 
                          on='station', how='left')

# Remove duplicates from the combined DataFrame
station_counts = station_counts.drop_duplicates().reset_index()
station_counts = station_counts.drop(columns=['index'])
station_counts = station_counts[['station','name','maxdate','mindate','count','longitude','latitude','elevation']]

# Write the resulting DataFrame to a CSV file
csv_filename = 'Outputs/station_counts.csv'
station_counts.to_csv(csv_filename, index=False)

station_counts.head()

,station,name,maxdate,mindate,count,longitude,latitude,elevation
0,GHCND:USC00417079,"PLAINVIEW, TX US",2023-08-01,1908-01-01,1196,-101.70230,34.18910,1027.2
1,GHCND:USW00013966,"WICHITA FALLS MUNICIPAL AIRPORT, TX US",2023-07-31,1897-01-01,1195,-98.49298,33.97855,308.6
2,GHCND:USC00414382,"HUNTSVILLE, TX US",2023-08-01,1903-01-01,1194,-95.54210,30.70650,150.6
3,GHCND:USC00418274,"SHERMAN, TX US",2023-07-31,1897-05-01,1186,-96.64190,33.70330,231.6
4,GHCND:USC00415429,"LULING, TX US",2023-06-30,1897-01-01,1183,-97.65770,29.67550,121.9


In [36]:
station_counts.shape

(916, 7)